In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

In [7]:
file = "UCD_TOT_00509"

In [8]:
import struct


def parse_fixed_width_string(text,fieldwidths):
    
    fmtstring  = ' '.join('{}{}'.format(abs(fw), 'x' if fw < 0 else 's')
                            for fw in fieldwidths)
    
    fieldstruct = struct.Struct(fmtstring)
    
    fileds = tuple(s.decode().strip() for s in fieldstruct.unpack_from(text.encode()) )
    
    return( fileds )

Description du format de lecture de chaque ligne et initialisation des objets utilisés dans la boucle de lecture

In [9]:
lists= {}
#Description des format et initialisation des listes qui seront remplies par la lecture du fichier
formats={}

formats["000  "] = ( 3, 2, 14, 6, 2, 14, 6, 2, 3, 8, 24, 5, 1, 4, 34 )  
formats["00101"] = tuple( [ 3, 2, 2 ] + [ 1, 8, 8, 3 ] * 5  + [ 21 ] )
formats["00201"] = formats["00101"]
formats["00301"] = tuple( [ 3, 2, 2 ] + [ 1, 8, 8, 4 ] * 5  + [ 16 ] )
formats["00401"] = tuple( [ 3, 2, 2 ] + [ 1, 8, 8, 7 ] * 4  + [ 25 ] )
formats["00501"] = formats["00301"]
formats["00601"] = tuple( [ 3, 2, 2 ] + [ 7, 8, 8 ] * 4  + [ 29 ] )
formats["00701"] = tuple( [ 3, 2, 2 ] + [ 3, 8, 8 ] * 5  + [ 26 ] )


#1010101 general informations about the ucd code
formats["1010101"] = ( 3, 2, 2, 13, 13, 13, 1, 30, 8, 25, 8, 10 )  
ucd = [] #  ucd code 7 characters
ucd_comp = {} # others ucd codes (13 characters,...)
company = [] # Short name of the company that commercialize the drug


formats["10102"] = ( 3, 2, 2, 100, 21 ) 

#10103 Conditionnement 
formats["10103"] = formats["10102"]

#10104 Observations
formats["10104"] = formats["10102"]

#102 technical informations about ucd

hospital_delivery = {} # list of drug that can only be delivered in a hospital  ( liste de la réserve hospitalière)
hospital_precription = {} # list of drug that can only be prescribed in a hospital 
hospital_first_precription = {} # list of drug  which first prescription can only be in a hospital 
special_monitoring = {} # list of drug  for which a special monotoring is mandatory
specialist_precription = {} # list of drug  for which the prescriptor can only be a specialist

formats["1020101"] = ( 3, 2, 2, 2, 1, 1, 1, 1, 1, 36, 78 ) 


#1020201 + 1020202 + 1020203 Libellé long RCP Médicament 1
display = {}

formats["1020201"] = formats["10102"]
formats["1020202"] = formats["10102"]
formats["1020203"] = formats["10102"]

#1020301 Formes compléments
formats["1020301"] = ( 3, 2, 2, 10, 40, 10, 40, 21 ) 

#1020401 Dosages Médicament
dosages = {}
formats["1020401"] = ( 3, 2, 2, 17, 17, 80, 7 ) 

#1020501 Classe ATC Médicament
atcs = {}
formats["1020501"] = ( 3, 2, 2, 10, 77, 34 ) 

#1020601 Classe EPHMRA Médicament
ephmras = {}
formats["1020601"] = formats["1020501"]

#11001 Historique périodes validités
dates_validity= {}
formats["11001"] = ( 3, 2, 2, 8, 8, 1, 8, 25, 10, 61 )

#12001 Historique tarifaire
formats["12001"] = ( 3, 2, 2, 8, 1, 8, 7, 1, 1, 1, 7, 8, 25, 54 ) 

#13001 Historique taux
formats["13001"] = ( 3, 2, 2, 8, 1, 1, 3, 8, 25, 75 ) 

#14001 Historique majorations
formats["14001"] = ( 3, 2, 2, 8, 1, 3, 4, 8, 25, 72 ) 

#15001 Historique coûts Supplémentaires
formats["15001"] = ( 3, 2, 2, 8, 1, 8, 8, 25, 71 ) 

#16001 Historiques des Tops Établissement
formats["16001"] = ( 3, 2, 2, 8, 1, 1, 8, 8, 25, 70 ) 

#17001 Historiques des Alertes Quantités Max
formats["17001"] = ( 3, 2, 2, 8, 1, 3, 5, 8, 8, 25, 63 ) 

#18001 Historiques des PECP
formats["18001"] = ( 3, 2, 2, 8, 1, 2, 8, 8, 25, 69 )

#2010101 Données CIP 
formats["2010101"] = ( 3, 2, 2, 13, 1, 107 ) 

#210101 Historiques des Périodes de lien du CIP
formats["21001"] = tuple( [ 3, 2, 2 ] + [ 8, 8 ] * 5  + [ 41 ] )
#2990101 Fin données CIP
formats["29901"] = ( 3, 2, 2, 5, 116 ) 

#1990101 ligne de fin données med x 
formats["1990101"] = tuple( [ 3, 2, 2 ] + [ 5 ] * 12 + [ 61 ] ) 

#Ligne de fin de fichier
formats["999"] = ( 3, 2, 14, 6, 2, 14, 6, 2, 3, 8, 8, 8, 10, 42 )

Lecture du fichier

In [10]:
#compteurs
c_line = 0; c_med = 0;


with open(file, 'r', encoding="latin1") as f:

        
        for line in f:
                
            line_format = re.search(r'^('+('|').join( [(x) for x in formats ])+')',line)[0]

            fields =  parse_fixed_width_string( line, formats[ line_format ] ) 
            
            #1010101 Début med +détail ucd
            if line_format == "1010101":
                
                c_med = c_med + 1
                
                ucd_active = fields[3].strip("0")
                
                ucd.append(ucd_active)
                
                company.append(fields[7])
                
                display[ucd_active] = None
                lists[ucd_active] = None


            #10102 Désignation
                            
            if line_format == "10102":
                display[ucd_active] = ' '.join( filter(None, ( display[ucd_active] , fields[3]  ) ) ) 
                
            #10103 Conditionnement 
            #10104 Observations
            #1020101 Prescriptions, Réserves, surveillance
            #1020201 + 1020202 + 1020203 Libellé long RCP Médicament 1
            #1020301 Formes compléments
            #1020401 Dosages Médicament
            
            #1020501 Classe ATC Médicament
            #1020601 Classe EPHMRA Médicament
            #11001 Historique périodes validités
            if line_format == "11001":

                    if fields[5] == "1":
                        if lists[ucd_active] is None:
                            lists[ucd_active]=[]
                            
                        lists[ucd_active].append({"billing_outpatient",fields[3] , fields[4] })
                        
                    if fields[5] == "2":
                        
                        if lists[ucd_active] is None:
                            lists[ucd_active]=[]
                        
                        lists[ucd_active].append({"billing_hospital",fields[3] , fields[4] })
                                                              
            #12001 Historique tarifaire
            #13001 Historique taux
            #14001 Historique majorations
            #15001 Historique coûts Supplémentaires
            #16001 Historiques des Tops Établissement
            #17001 Historiques des Alertes Quantités Max
            #18001 Historiques des PECP
            #2010101 Données CIP 
            #210101 Historiques des Périodes de lien du CIP
            #2990101 Fin données CIP
            #1990101 Fin med
            
                        


In [16]:
medicaments=pd.DataFrame({'ucd': pd.Series(ucd,index=ucd),
                          'company':pd.Series(company,index=ucd),
                   'display' : pd.Series(display),
                   'lists': pd.Series(lists)
                         })

In [17]:
medicaments

,ucd,company,display,lists
9009994,9009994,PROSTRAKAN PHARMA,BELUSTINE 40 mg,"[{00000000, billing_outpatient, 20050511}]"
901984,901984,AVENTIS,CISPLATYL 25 mg,"[{00000000, billing_outpatient, 20050511}]"
9019857,9019857,AVENTIS,CISPLATYL 50 mg,"[{00000000, billing_outpatient, 20050511}]"
9024114,9024114,PFIZER,DEBRIDAT 50 mg/5 ml,"[{billing_outpatient, 20190221, 20051231}]"
9025711,9025711,AVENTIS,DETICENE 100 mg,"[{00000000, billing_outpatient, 20050511}]"
...,...,...,...,...
9454636,9454636,ACCORD HEALTHCARE FRANCE SAS,POSACONAZ.AHCL40MG/ML,"[{00000000, 20200313, billing_outpatient}]"
945482,945482,EG LABO LABORATOIRES EUROGENER,POSACONAZOLE EG 100MG,"[{00000000, billing_outpatient, 20200219}]"
9455506,9455506,ACCORD HEALTHCARE FRANCE SAS,AZACITIDINE ACC 25MG/ML,"[{00000000, billing_outpatient, 20200418}, {00..."
9456227,9456227,CEMAG CARE,IVOZALL 1MG/ML PERF,"[{00000000, billing_hospital, 20200410}]"
